<a href="https://colab.research.google.com/github/AffanSurya/SenakApp_ML/blob/main/Model/RecomandationsS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install tensorflow_recommenders

In [23]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text

import warnings
warnings.filterwarnings('ignore')

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
df_recipes = pd.read_csv('/content/drive/MyDrive/Dataset/recipes_reduce.csv')
df_reviews = pd.read_csv('/content/drive/MyDrive/Dataset/reviews_reduce.csv')

In [26]:
df_recipes.info()
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23899 entries, 0 to 23898
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    23899 non-null  int64  
 1   Name                        23899 non-null  object 
 2   AuthorId                    23899 non-null  int64  
 3   Description                 23899 non-null  object 
 4   Images                      23899 non-null  object 
 5   RecipeCategory              23891 non-null  object 
 6   RecipeIngredientQuantities  23899 non-null  object 
 7   RecipeIngredientParts       23899 non-null  object 
 8   AggregatedRating            23898 non-null  float64
 9   ReviewCount                 23899 non-null  float64
 10  Calories                    23899 non-null  float64
 11  Fat                         23899 non-null  float64
 12  SaturatedFat                23899 non-null  float64
 13  Cholesterol                 238

In [27]:
df_recipes.head()

,RecipeId,Name,AuthorId,Description,Images,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,Fat,SaturatedFat,Cholesterol,Sodium,Carbohydrate,Fiber,Sugar,Protein,RecipeInstructions
0,40,Best Lemonade,1566,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ..."
1,42,Cabbage Soup,1538,Make and share this Cabbage Soup recipe .,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil..."
2,44,Warm Chicken A La King,1596,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,23.0,895.5,66.8,31.9,405.8,557.2,29.1,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co..."
3,49,Chicken Breasts Lombardi,174711,Make and share this Chicken Breasts Lombardi r...,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""2"", ""2"", ""12"", ""1/2"", ""1/3"", ""3/4"", ""1/2"", ...","c(""fresh mushrooms"", ""butter"", ""boneless skinl...",5.0,21.0,627.7,24.9,13.0,203.0,848.9,13.7,0.6,2.0,57.9,"c(""Cook mushrooms in 2 tbsp butter in a large ..."
4,54,Carrot Cake,1535,This is one of the few recipes my husband ever...,"c(""https://img.sndimg.com/food/image/upload/w_...",Dessert,"c(""3"", ""4"", ""1 1/4"", ""2"", ""2"", ""2"", ""2"", ""1"", ...","c(""carrots"", ""eggs"", ""white sugar"", ""all-purpo...",5.0,17.0,522.6,27.1,4.9,69.8,534.8,67.0,1.6,47.9,5.0,"c(""Beat together the eggs, oil, and white suga..."


In [28]:
df_reviews.head()

,ReviewId,Name,RecipeId,AuthorId,Rating,Review,DateSubmitted,DateModified
0,2,Jalapeno Pepper Poppers,992,2008,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,13,Kevin's Best Corned Beef,7435,1773,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
2,14,Warm Chicken A La King,44,2085,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
3,17,Spinach and Strawberry Salad,5221,2046,4,"love it, but without the bean sprouts.",2000-05-08T11:08:00Z,2000-05-08T11:08:00Z
4,19,Neiman-Marcus $250 Chocolate Chip Cookies Recipe,13307,2046,5,"chewy goodness, not crispy at all. i even thre...",2000-05-21T16:59:00Z,2000-05-21T16:59:00Z


In [29]:
df_reviews = df_reviews.astype({'AuthorId': 'string'})


Reviews = tf.data.Dataset.from_tensor_slices(dict(df_reviews[['AuthorId', 'Name', 'Rating']]))
Recipes = tf.data.Dataset.from_tensor_slices(dict(df_recipes[['Name']]))

Reviews = Reviews.map(lambda x: {
    "Name": x["Name"],
    "AuthorId": x["AuthorId"],
    "Rating": float(x["Rating"])
})

Recipes = Recipes.map(lambda x: x["Name"])

In [30]:
print('Total Data: {}'.format(len(Reviews)))

tf.random.set_seed(42)
shuffled = Reviews.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = Reviews.take(35_000)
test = Reviews.skip(35_000).take(8_188)

Total Data: 724541


In [31]:
food_titles = Recipes.batch(1_000)
user_ids = Reviews.batch(1_000).map(lambda x: x["AuthorId"])

unique_food_titles = np.unique(np.concatenate(list(food_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Recipes: {}'.format(len(unique_food_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Recipes: 23899
Unique users: 185252


In [32]:
class FoodModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    self.food_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_food_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_food_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=Recipes.batch(128).map(self.food_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    user_embeddings = self.user_model(features["AuthorId"])
    food_embeddings = self.food_model(features["Name"])

    return (
        user_embeddings,
        food_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        self.rating_model(
            tf.concat([user_embeddings, food_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("Rating")

    user_embeddings, food_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, food_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [114]:
model = FoodModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 124s 3s/step - root_mean_squared_error: 1.8080 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0022 - factorized_top_k/top_10_categorical_accuracy: 0.0045 - factorized_top_k/top_50_categorical_accuracy: 0.0154 - factorized_top_k/top_100_categorical_accuracy: 0.0247 - loss: 6909.8831 - regularization_loss: 0.0000e+00 - total_loss: 6909.8831
Epoch 2/3
35/35 [==============================] - 112s 3s/step - root_mean_squared_error: 1.0067 - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.0717 - factorized_top_k/top_10_categorical_accuracy: 0.1097 - factorized_top_k/top_50_categorical_accuracy: 0.2503 - factorized_top_k/top_100_categorical_accuracy: 0.3521 - loss: 6728.7809 - regularization_loss: 0.0000e+00 - total_loss: 6728.7809
Epoch 3/3
35/35 [==============================] - 112s 3s/step - root_mean_squared_error: 1.0926 - 

In [34]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 28s 3s/step - root_mean_squared_error: 0.9768 - factorized_top_k/top_1_categorical_accuracy: 2.4426e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0012 - factorized_top_k/top_10_categorical_accuracy: 0.0029 - factorized_top_k/top_50_categorical_accuracy: 0.0137 - factorized_top_k/top_100_categorical_accuracy: 0.0243 - loss: 5832.4316 - regularization_loss: 0.0000e+00 - total_loss: 5832.4316

Retrieval top-100 accuracy: 0.024
Ranking RMSE: 0.977


In [126]:

random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def predict_food_for_random_user(top_n=5):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
      tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

predict_food_for_random_user(top_n=5)


Top 5 recommendations for user 637765:

1. Belly Buster
2. Pancakes
3. Burrito Pie
4. Should Be Illegal Oven BBQ Ribs
5. Impossible Peanut Butter Cookies


In [94]:
def get_image_url(recipe_df, recipe_name, image_column='Images'):
    # Mengambil URL pertama dari setiap baris dalam kolom 'Images'
    image_urls = recipe_df[recipe_df['Name'] == recipe_name][image_column].values
    if len(image_urls) > 0:
        # Menggunakan ekspresi reguler untuk mendapatkan URL yang benar
        match = re.search(r'https://.*?\.jpg', image_urls[0])
        if match:
            return match.group()
    return None


def predict_food_for_random_user_with_images(recipe_df, top_n=5):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
        tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        title_str = title.decode("utf-8")

        # Get the image URL for the recommended food using the function
        image_url = get_image_url(recipe_df, title_str)

        print('{}. {}\n   Image: {}'.format(i+1, title_str, image_url))

# Panggil fungsi dengan DataFrame Anda
predict_food_for_random_user_with_images(df_recipes, top_n=5)


Top 5 recommendations for user 353579:

1. Fudge Crinkles (A Great 4 Ingredient Cake Mix Cookie)
   Image: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/32/61/4/y8M4HL6NRWywP92X5Hky_0S9A1122.jpg
2. Chicken Stroganoff - Crock Pot
   Image: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/18/65/1/OMDshWgTQ4yutIk4fHS0_broccoliandchicken0069.jpg
3. Potato Kielbasa Skillet
   Image: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/34/68/8/UnxcCWnsQB62X5zZXU60_DSC_0287.jpg
4. Crock Pot Lasagna
   Image: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/21/70/6/q9xSCChZQNSujJG2TNsF_lasagna%202%20(1%20of%201).jpg
5. Top Rated Classic Meatloaf
   Image: https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/15/02/7/X0KAo2nGTHW6QbqoUvbQ_MEATLOAF_023.jpg


In [ ]:
# Load the saved weights
model.load_weights('/content/drive/MyDrive/Dataset/food_recommendation_model.h5')

In [127]:
export_dir = "/content/drive/MyDrive/Capstone/test_saved_model"

# Wrap the relevant parts in tf.function
@tf.function(input_signature=[input_features])
def serving_fn(inputs):
    return model.call(inputs)

# Save the model in SavedModel format
tf.saved_model.save(model, export_dir, signatures={'serving_default': serving_fn})

# Convert the SavedModel to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Save the TF Lite model
tflite_model_file = '/content/drive/MyDrive/Capstone/food_recommendation_model.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)


FailedPreconditionError: ignored